# HW5: Image Captioning Tests
---

This is the Test Notebook that goes with **Homework 5: Image Captioning**! 

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from transformer import *
from decoder import *

### Part 2

In [3]:
# Test RNN Decoder
# DO NOT CHANGE

tf.random.set_seed(42)

input_size = 4
window_size = 16
embed_size = 8
vocab_size = 2

SOLUTION_SHAPE = (input_size, window_size, vocab_size)

hidden_size = 32

rnn_decoder = RNNDecoder(vocab_size, hidden_size, window_size)
out = rnn_decoder(
    tf.random.uniform([input_size, 64], minval=0, maxval=vocab_size),
    tf.random.uniform([input_size, window_size], maxval=vocab_size)
)
epsilon = 1e-5
assert out.shape == SOLUTION_SHAPE, "Incorrect output shape"
assert not (tf.reduce_sum(out) > input_size * window_size - epsilon and tf.reduce_sum(out) < input_size * window_size + epsilon), "Return logits, not probabilities"
print("Tests passed!")

Saved
Tests passed!


In [5]:
# Test Transformer Decoder
# DO NOT CHANGE

tf.random.set_seed(42)

input_size = 4
window_size = 16
embed_size = 8
vocab_size = 2

SOLUTION_SHAPE = (input_size, window_size, vocab_size)

hidden_size = 32

transformer_decoder = TransformerDecoder(vocab_size, hidden_size, window_size)
out = transformer_decoder(
    tf.random.uniform([input_size, 64], minval=0, maxval=vocab_size),
    tf.random.uniform([input_size, window_size], maxval=vocab_size)
)
epsilon = 1e-5
assert out.shape == SOLUTION_SHAPE, "Incorrect output shape"
assert not (tf.reduce_sum(out) > input_size * window_size - epsilon and tf.reduce_sum(out) < input_size * window_size + epsilon), "Return logits, not probabilities"
print("Tests passed!")

Tests passed!


## Part 3

In [163]:
# Test AttentionHead call function
# DO NOT CHANGE

tf.random.set_seed(42)

SOLUTION = np.array([
 [[1.1887338,  0.92231,    1.1832824,  1.4413241,  0.9284028,  1.2374643,
   1.2092555,  1.6531762, ],
  [1.1876637,  0.920028,   1.1839285,  1.4362004,  0.92912036, 1.2364106,
   1.208142,   1.6493324, ],
  [1.1947478,  0.92511594, 1.18963,   1.4394116,  0.9363934,  1.2449214,
   1.2101097,  1.6523575, ],
  [1.177351,   0.9136951,  1.1745526,  1.4348177,  0.91724336, 1.2238563,
   1.2058568,  1.6473322, ]],

 [[1.171719,   0.91546506, 1.2353648,  1.2881929,  0.955095,   1.2792217,
   1.3040287,  1.6174344, ],
  [1.1937302,  0.9304202,  1.2462558,  1.3073134,  0.9750139,  1.3008424,
   1.2973801,  1.6244173, ],
  [1.1734747,  0.9167778,  1.2366297,  1.2896433,  0.9566818,  1.281148,
   1.304501,   1.618589,  ],
  [1.204712,   0.9370769,  1.2490294,  1.3173547,  0.9849655,  1.3102754,
   1.2873328,  1.6238908, ]]])

def create_deterministic_attention_head(input_size, output_size, is_self_attention):
    head = AttentionHead(input_size, output_size, is_self_attention)

    head.K = tf.random.uniform([input_size, output_size]) # This is not correct. This is for testing purposes only
    head.V = tf.random.uniform([input_size, output_size]) # This is not correct. This is for testing purposes only
    head.Q = tf.random.uniform([input_size, output_size]) # This is not correct. This is for testing purposes only

    return head

input_size = 4
batch_size = 2
window_size_keys = 3
window_size_values = 3
window_size_queries = 4

head = create_deterministic_attention_head(4, 8, False)

out = head.call(
    tf.random.uniform([batch_size, window_size_keys, input_size], dtype=np.float32),
    tf.random.uniform([batch_size, window_size_values, input_size], dtype=np.float32),
    tf.random.uniform([batch_size, window_size_queries, input_size], dtype=np.float32)
).numpy()

assert out.shape == SOLUTION.shape, "Incorrect output shape"
try: assert np.allclose(out, SOLUTION, rtol=0.001), "Incorrect output values"
except AssertionError:
    print("Output:")
    print(out)
    print("Solution:")
    print(SOLUTION)
    assert False, "Incorrect output values"
print("Tests passed!")

Tests passed!


In [4]:
# Test TransformerBlock
# DO NOT CHANGE

tf.random.set_seed(42)

SOLUTION = np.array([
 [[0.         ,0.6974105  ,0.98577017 ,0.        ],
  [0.         ,0.31697747 ,1.4552947  ,0.        ],
  [0.         ,0.         ,1.6302229  ,0.        ],
  [0.8908619  ,0.761421   ,0.         ,0.        ],
  [0.         ,1.093846   ,0.         ,0.85964626],
  [0.         ,0.         ,1.1071919  ,0.84442014],
  [0.         ,0.7972666  ,0.         ,1.1266519 ],
  [0.         ,0.         ,1.6696594  ,0.        ]],

 [[0.         ,0.         ,1.6952605  ,0.        ],
  [0.         ,1.040991   ,0.9518903  ,0.        ],
  [0.         ,0.9950613  ,0.19130549 ,0.47112876],
  [0.         ,1.2255263  ,0.72682804 ,0.        ],
  [0.59422725 ,0.         ,1.3293806  ,0.        ],
  [1.56447    ,0.         ,0.         ,0.01488148],
  [0.         ,0.6034657  ,0.         ,1.1571136 ],
  [0.         ,0.18686305 ,0.29973274 ,1.1265295 ]]])

batch_size = 2
input_seq_length = 8
embedding_size = 4
context_seq_length = 6
multiheadedattention = False

transformer_block = TransformerBlock(embedding_size, multiheadedattention)
transformer_block.self_atten = create_deterministic_attention_head(embedding_size, embedding_size, True)  # This is not correct. This is for testing purposes only
transformer_block.self_context_atten = create_deterministic_attention_head(embedding_size, embedding_size, True)  # This is not correct. This is for testing purposes only
transformer_block.ff_layer = tf.keras.layers.Dense(1, kernel_initializer=tf.keras.initializers.constant()) #  # This is not correct. This is for testing purposes only

out = transformer_block(
    tf.random.uniform([batch_size, input_seq_length, embedding_size]),
    tf.random.uniform([batch_size, context_seq_length, embedding_size])
).numpy()

assert out.shape == SOLUTION.shape, "Incorrect output shape"
assert not (out < 0).any(), "Incorrect output activation"
try: assert np.allclose(out, SOLUTION, rtol=0.001), "Incorrect output values"
except AssertionError:
    print("Output:")
    print(out)
    print("Solution:")
    print(SOLUTION)
    assert False, "Incorrect output values"
print("Tests passed!")

NameError: name 'create_deterministic_attention_head' is not defined

In [164]:
# Test Positional Encoding

tf.random.set_seed(42)

SOLUTION = np.array([
 [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
   1.0000000e+00,  1.0000000e+00,  1.0000000e+00,  1.0000000e+00],
 [ 8.4147096e-01,  9.9833414e-02,  9.9998331e-03,  9.9999981e-04,
   5.4030228e-01,  9.9500418e-01,  9.9994999e-01,  9.9999952e-01],
 [ 9.0929741e-01,  1.9866933e-01,  1.9998666e-02,  1.9999987e-03,
  -4.1614684e-01,  9.8006660e-01,  9.9980003e-01,  9.9999797e-01],
 [ 1.4112000e-01,  2.9552022e-01,  2.9995501e-02,  2.9999956e-03,
  -9.8999250e-01,  9.5533651e-01,  9.9955004e-01,  9.9999553e-01]])

input_size = 4
window_size = 4
embed_size = 8
vocab_size = 2

dummy_layer = tf.keras.layers.Dense(embed_size, kernel_initializer=tf.keras.initializers.constant())

positional_encoding = PositionalEncoding(vocab_size, embed_size, window_size)
positional_encoding.embedding = dummy_layer  # This is not correct. This is for testing purposes only

out = positional_encoding.call(tf.random.uniform([input_size, window_size])).numpy()

assert out.shape == SOLUTION.shape, "Incorrect output shape"
try: assert np.allclose(out, SOLUTION, rtol=0.001), "Incorrect output values"
except AssertionError:
    print("Output:")
    print(out)
    print("Solution:")
    print(SOLUTION)
    assert False, "Incorrect output values"
print("Tests passed!")

Tests passed!
